# Quantum Simulator Project Report
## By: Francisco, Mike, and Troy

# Introduction

This project shows how to implement a local quantum simulator using only algorithms and Numpy function to simulate what a real quantum simulator would look like. The goal of the project was to create a CLI for our simulator and to be able to run any algorithms as long as they follow our input format specified in our read me. 

# Our Algorithm Design

You can see all the source code in the simulator.py which has 2 classes: CircuitParser and QuantumSimulator. These two classes work side by side. Our CiruitParser parses our inputs and returns the qubits, gates, and measurements. We then take this data and put it into our QuantumSimulator to actually run the quantum algorithms. 

In [2]:
class QuantumSimulator:
    # Declare all the aspects of our quantim simulator
    def __init__(self, num_qubits, noise_prob=0.0):
        self.num_qubits = num_qubits
        self.num_states = 2 ** num_qubits
        self.single_qubit_error = noise_prob
        self.double_qubit_error = noise_prob * 10
        self.state_vector = np.zeros(self.num_states, dtype=complex)
        self.state_vector[0] = 1.0

        print(f"Initialized {num_qubits}-qubit system")
        print(f"State vector size: {self.num_states}")
        print(f"Initial state: |{'0' * num_qubits}>")
    # Return the state of the vector
    def get_state_vector(self):
        return self.state_vector.copy()
    # Apply a single qubit gate
    # We use this in our gate applications to apply the gates we designed
    def apply_single_qubit_gate(self, gate_matrix, target_qubit):
        n = self.num_qubits

        state = self.state_vector.reshape([2] * n)

        state = np.moveaxis(state, target_qubit, -1)

        state = np.tensordot(state, gate_matrix, axes=[[-1], [0]])

        state = np.moveaxis(state, -1, target_qubit)

        self.state_vector = state.flatten()
    # We apply the x gate by creating a complex array and using our apply
    # single qubit gate
    def apply_X_gate(self, target_qubit):
        X = np.array([[0,1],
                     [1,0]], dtype=complex)
        self.apply_single_qubit_gate(X, target_qubit)
        #print(f"Applied X gate to qubit {target_qubit}")
    # We do the same above for H gate
    def apply_H_gate(self, target_qubit):
        H = (1/np.sqrt(2)) * np.array([[1, 1],
                                       [1, -1]], dtype=complex)
        self.apply_single_qubit_gate(H, target_qubit)
        #print(f"Applied H gate to qubit {target_qubit}")
    # We do the same above for our CNOT
    def apply_CNOT_gate(self, control_qubit, target_qubit):
        new_state = np.zeros_like(self.state_vector)
        for i in range(self.num_states):
            binary = format(i, f'0{self.num_qubits}b')
            bits = [int(b) for b in binary]

            if bits[control_qubit] == 1:
                bits[target_qubit] = 1 - bits[target_qubit]
            new_index = int(''.join(map(str, bits)), 2)
            new_state[new_index] = self.state_vector[i]
        self.state_vector = new_state
        #print(f"Applied CNOT gate: control={control_qubit}, target={target_qubit}")
    # This is how we actually apply the gates we pull from our parser
    def apply_gate(self, gate):
        gate_type = gate['type']
        qubits = gate['qubits']

        if gate_type == 'X':
            self.apply_X_gate(qubits[0])
            self.apply_bit_flip_noise([qubits[0]], is_two_qubit=False)
        elif gate_type == 'H':
            self.apply_H_gate(qubits[0])
            self.apply_bit_flip_noise([qubits[0]], is_two_qubit=False)
        elif gate_type == 'CNOT':
            self.apply_CNOT_gate(qubits[0], qubits[1])
            self.apply_bit_flip_noise([qubits[0], qubits[1]], is_two_qubit=True)
        else: 
            print(f"Warning: Unknown gate type {gate_type}")
    # We measure our qubits
    def measure(self, qubits_to_measure):
        probabilities = np.abs(self.state_vector) ** 2

        num_shots = 1000
        results = {}

        print(f"Measuring qubits {qubits_to_measure} ({num_shots} shots)")

        # Measure the qubits for the amount of shots in the range of our total shots
        for shot in range(num_shots):
            measured_state_index = np.random.choice(
                self.num_states,
                p=probabilities
            )
            # We put our bits back into binary format
            full_binary = format(measured_state_index, f'0{self.num_qubits}b')

            measured_bits = ''.join([full_binary[q] for q in qubits_to_measure])
            # If the measured bit is in the results then we add by one else we set it to 1
            if measured_bits in results:
                results[measured_bits] += 1
            else: 
                results[measured_bits] = 1
            
        return results
    # Here we just measure all the qubits
    def measure_all(self):
        return self.measure(list(range(self.num_qubits)))
    # Here is where we apply quantum noise by using a random probability set in our noise_prob
    # By default it is set to 0.0
    def apply_bit_flip_noise(self, affected_qubits, is_two_qubit=False):
        error = self.double_qubit_error if is_two_qubit else self.single_qubit_error

        if error == 0.0:
            return

        for qubit in affected_qubits:
            if np.random.random() < error:
                print(f"Noise bit flip on qubit {qubit}")
                temp_noise = self.single_qubit_error
                self.single_qubit_error = 0.0
                self.apply_X_gate(qubit)
                self.single_qubit_error = temp_noise
         
    def print_state(self):
        print("\n Current Quantum State:")
        for i, amplitude in enumerate(self.state_vector):
            if abs(amplitude) > 1e-10:
                binary_state = format(i, f'0{self.num_qubits}b')
                print(f"|{binary_state}> : {amplitude:.4f}")



As we can see from the code above there are several functions within this QuantumSimulator class. They all are extremely important to our functioning algorithm. Below I will outline what each function does and why it is important

## Functions
- get_state_vector: returns a copy of the state vector which we then use in our functions below.
- apply_single_qubit_gate(self, gate_matrix, target_qubit): Discuss this 
- apply_X_gate(self, target_qubit): Here we apply our X gate by creating it through a numpy array with a complex data type and then pass this into our apply_single_qubit_gate
- apply_H_gate(self, target_qubit): Here we do the same as X gate except with an H gate matrix
- apply_CNOT_gate(self, control_qubit, target_qubit): We do essentially create the binary representations of our target_qubit and control_qubit, and then create a new vector state to apply the CNOT gate by directly changing our state_vector indicies instead of using apply_single_qubit_gate as it is more optimal than our 2^n application in the previous gates.
- apply_gate(self, gate): Here we get the gate type and then use if statements to apply the gates based on the gate given into the function
- measure(self, qubits_to_measure): This simulates the collapse of a quantum state when we measure specific qubits. We do this through 7 steps. Step 1 is to calculate the probabilites for each state and we do this by taking the magnitude of the state vector and sqauring it. Then we perform the measurements in our case it is 1000 shots. We then iterate through each shot and measure the index by taking np.random.choice between the number of states and the probabilites which gives us the measurement_index. After this we convert the index to a binary string, and then extract only the measure qubits. Then we count the results. Thats how we get to our final result which gives us the amount of times each result shows. 
- measure_all(self): This measures all the qubits
- apply_noise(self): Here we apply quantum noise by taking the probability of noise and comparing it to a random number and if the noise is greater than we apply an x_gate to simulate a random bit flip
- print_state(self): Here we print our states

## Performance Evaluation

Here we use our Benchmark.py to run the performance analyzation of our results, the code block below imports our important functions from our benchmark.py and reports the results of the benchmark

In [1]:
from benchmark import benchmark_circuit_file, benchmark_gates, benchmark_qubits
import os
print("Quantum Simulator Performance Benchmarks")

print("\n\nStarting benchmarks:")

qubit_results = benchmark_qubits()
gate_results = benchmark_gates()

test_circuits = [
        'test_bell.in',
        'test_circuit',
        'test_ghz.in'
]

print("Benchmark 3: Test circuit performance")

for circuit in test_circuits:
    if os.path.exists(circuit):
        benchmark_circuit_file(circuit)
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print("\nComplexity Analysis:")
print("- State vector size: O(2^n) where n = number of qubits")
print("- Single-qubit gate: O(2^n) matrix-vector multiplication")
print("- Two-qubit gate: O(2^n) state transformation")
print("- Memory usage: O(2^n) complex numbers")
print("\nScalability:")
if len(qubit_results) >= 2:
    # Compare last two measurements
    n1, states1, time1, mem1 = qubit_results[-2]
    n2, states2, time2, mem2 = qubit_results[-1]
    ratio = time2 / time1
    print(f"- Going from {n1} to {n2} qubits: ~{ratio:.1f}x slower")
    print(f"- Memory grows exponentially: {mem2/mem1:.1f}x increase")

Quantum Simulator Performance Benchmarks


Starting benchmarks:
Benchmark 1: Runtime vs Number of Quibits
Circuit: apply H gate to all qubits
Initialized 3-qubit system
State vector size: 8
Initial state: |000>
Measuring qubits [0, 1, 2] (1000 shots)
Qubits: 3          Possible Qubit Comb: 8               Elapsed: 1.0582       Memory Used:0.00        
Initialized 5-qubit system
State vector size: 32
Initial state: |00000>
Measuring qubits [0, 1, 2, 3, 4] (1000 shots)
Qubits: 5          Possible Qubit Comb: 32              Elapsed: 0.0169       Memory Used:0.00        
Initialized 7-qubit system
State vector size: 128
Initial state: |0000000>
Measuring qubits [0, 1, 2, 3, 4, 5, 6] (1000 shots)
Qubits: 7          Possible Qubit Comb: 128             Elapsed: 0.0165       Memory Used:0.00        
Initialized 10-qubit system
State vector size: 1024
Initial state: |0000000000>
Measuring qubits [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] (1000 shots)
Qubits: 10         Possible Qubit Comb: 1024          

## Results

As we can see from above, we have results from 3 distinct benchmarks. 

### Benchmark 1: Runtime vs Number of Qubits

Fill out

### Benchmark 2: Runtime vs Number of Gates

Here we initialized a 10 qubit system and tested it against up 100 gates and we can see that the system increases linearly in time. The amount of data being processed is not nearly as bad as increasing the amount of qubits. With all of our tests finishing in below 1 second. It goes to show that the number of gates does not nearly affect the size or time complexity nearly as badly as the number of qubits.

### Benchmark 3: Test circuit performance

Our test circuits all ran in under 0.004 seconds. This is very good and makes sense considering we only had 2 gates and 2 qubits for our bell state circuit and 3 gates and 3 qubits for our ghz circuit test. We can see that our circuit parser and quantum simulator are working as expected.

### Summary

All in all our simulator is working as intended and the algorithms are able to handle qubits up until we get to 26 qubits due to the sheer size complexity we hit once we have that many qubits. Our algorithm is running quite quick and hopefully we can improve this in the future to handle more qubits.

## Noise Modeling

We modeled the noise for our simulator through adding a probability argument and testing it against a random number, and if this number was below our probability then we called an x gate and flipped the bit. This is very reasonable as this is similar to what happens with actual quantum noise. 
**Discuss the two_qubit_error rate here for CNOT**

#### Noiseless Result: test_bell.in:

Loading circuit: test_bell.in
Circuit has 2 qubits
Added gate: H on qubit 0
Added gate: CNOT on qubits 0, 1
Measure qubits: [0, 1]

Circuit Summary:
 Qubits: 2
 Gates: 2
 Measurements: [0, 1]
 Mode: Noiseless


Simulation
Initialized 2-qubit system
State vector size: 4
Initial state: |00>

Applying gates...

[Gate 1]/2
 Current Quantum State:
|00> : 0.7071+0.0000j
|10> : 0.7071+0.0000j

[Gate 2]/2
 Current Quantum State:
|00> : 0.7071+0.0000j
|11> : 0.7071+0.0000j


Final Gate

 Current Quantum State:
|00> : 0.7071+0.0000j
|11> : 0.7071+0.0000j
Measurement
Measuring qubits [0, 1] (1000 shots)

Measurement Results:
|00> : 494/1000 = 49.4%
|11> : 506/1000 = 50.6%


Results Summary


|00> :  494 █████████████████████████████████████████████████

|11> :  506 ██████████████████████████████████████████████████

#### Noisy Result Error rate of 0.2 to see the bit flip: 

Loading circuit: test_bell.in
Circuit has 2 qubits
Added gate: H on qubit 0
Added gate: CNOT on qubits 0, 1
Measure qubits: [0, 1]

Circuit Summary:
 Qubits: 2
 Gates: 2
 Measurements: [0, 1]

 Mode: Noise (error rate = 0.2)


Simulation
Initialized 2-qubit system
State vector size: 4
Initial state: |00>

Applying gates...

[Gate 1]/2 
 Current Quantum State:
|00> : 0.7071+0.0000j
|10> : 0.7071+0.0000j

[Gate 2]/2 [Noise] bit flip on qubit 0

 Current Quantum State:
|01> : 0.7071+0.0000j
|10> : 0.7071+0.0000j


Final Gate

 Current Quantum State:
|01> : 0.7071+0.0000j
|10> : 0.7071+0.0000j
Measurement
Measuring qubits [0, 1] (1000 shots)

Measurement Results:
|01> : 480/1000 = 48.0%
|10> : 520/1000 = 52.0%


Results Summary

|01> :  480 ████████████████████████████████████████████████

|10> :  520 ████████████████████████████████████████████████████

As we can see above the noise caused a bit flip, although we had to exagerate the error prob and We had to run it a handful of times to see this. This is what real quantum noise can look like and the bit flip shows why quantum noise is so important to understand. Now with a normal 1 percent error we won't see bit flips most of the time, which models real quantum hardware. 

# Conclusion

### Key Findings

All test circuits produce correct results, which match what should happen in theory. The simulator can handle up to 25 qubits efficiently before it becomes too much for the hardware on our laptops to handle. This is due to the exponential increase of the size complexity for qubits. For gates they scale linearly which means the simulator could handle up to 1000 gates in under 20 seconds, which shows that the gates are not nearly as tough on the hardware as the qubits are. 

**Noise Modeling**: We modeled noise through taking a random number and checking if it is less than the error probability. If this was the case we would flip the bit through our apply_x_gate function. Effectively simulating what real quantum noise would look like in an actual quantum simulator. 

### Limitations
- We ran into memory constraints when it came to the qubit bench mark. 
- **Frankie will fill out later**
- We only support 3 gates at the moment which are X, H, and CNOT which is the base for a quantum simulator but could do with more gate support. 

Over all the project is implemented to the best of our ability and was really interesting to see how to build a quantum simulator without quantum hardware. This experience taught us how these systems work under the hood and this report outlines how we went about our work. 